In [10]:
from eda import get_objectives, get_constraints, non_dominated_sort, non_dominated, assign_crowding_distance, binary_tournament_selection, sample_population, cleanupsamples, generate_example_data, organize_results

In [11]:
from numpy import random
import numpy as np
from scipy.stats import gamma, norm
import pandas as pd
from sklearn.linear_model import LinearRegression
from scipy.stats import multivariate_normal as mvn
from scipy.spatial.distance import jensenshannon
from numba import jit
import math
from hdf5storage import loadmat, savemat
import pickle

In [3]:
def transform_items_to_z(items):
    alpha = np.empty(items.shape[1])
    beta = np.empty(items.shape[1])
    items_z = np.empty(items.shape)
    for i in range(items.shape[1]):
        a, loc, scale = gamma.fit(items[:, i], floc=0.0)
        alpha[i] = a
        beta[i] = scale
        u = gamma.cdf(items[:,i], a = a, scale = scale)
        u = np.clip(u, 1e-12, 1-1e-12)
        items_z[:,i] = norm.ppf(u) 
    return alpha, beta, items_z

In [4]:
def convert_to_long(YY, n_selected, n_obj, n_con):
    rows = []
    for j in range(n_selected):
        for k in range(n_obj+n_con):
            if k < n_obj:
                rows.append(pd.DataFrame({
                    'cumulative': YY[:,j, k],
                    'step': f'Step {j}',
                    'objective': f'Obj {k}'
                }))
            else:
                rows.append(pd.DataFrame({
                    'cumulative': YY[:,j, k],
                    'step': f'Step {j}',
                    'objective': f'Con {k-n_obj}',
                }))
    df_Y = pd.concat(rows, ignore_index=True)
    return df_Y

def ecdf(df_table,n_selected,n_obj,n_con):
    nk = len(df_table)/(n_selected*(n_obj+n_con))
    ecdf_table = np.zeros((n_selected,(n_obj+n_con),int(nk)))
    for t in range(1,n_selected):
        for i in range(n_obj+n_con):
            if i < n_obj:
                ecdf_table[t,i,:] = np.sort(df_table['cumulative'][(df_table['objective'] == f'Obj {i}')&(df_table['step'] == f'Step {t}')].values)
            else:
                ecdf_table[t,i,:] = np.sort(df_table['cumulative'][(df_table['objective'] == f'Con {i - n_obj}')&(df_table['step'] == f'Step {t}')].values)
    return ecdf_table

def z_transform_from_ecdf(Y, ecdf_table):
    N, d = Y.shape  
    Y_z = np.empty_like(Y)
    for t in range(1,N):
        for i in range(d):
            ranks = np.searchsorted(ecdf_table[t,i,:], Y[t, i], side='right')
            u = ranks / len(ecdf_table[t,i,:])
            u = np.clip(u, 1e-12, 1 - 1e-12)
            Y_z[t, i] = norm.ppf(u) 
    return Y_z

In [5]:
def get_norm_cumu_objectives(items, items_z, population, n_selected, n_obj, n_con, rng, if_inital): 
# can modify to use objectives instead of population
    XX = np.empty((population.shape[0], n_selected, n_obj+n_con))
    XX_z = np.empty((population.shape[0], n_selected, n_obj+n_con))
    for k in range(population.shape[0]):
        if if_inital:
            qx = rng.permutation(population[k, :]) # permutation only for initial population
        else:
            qx = population[k, :]
        XX[k,:,:] = items[qx,:]
        XX_z[k,:,:] = items_z[qx,:]
    YY = np.cumsum(XX, axis = 1)
    # df_Y = convert_to_long(YY, n_selected, n_obj, n_con)
    # ecdf_table = ecdf(df_Y, n_selected, n_obj, n_con)
    # YY_z = np.empty_like(YY)
    # for k in range(YY.shape[0]):
    #     YY_z[k] = z_transform_from_ecdf(YY[k], ecdf_table)
    #     YY_z[k,0,:] = XX_z[k,0,:]
    YY_z = np.cumsum(XX_z, axis = 1)
    return XX, XX_z, YY, YY_z

In [22]:
def fit_markov_in_y_by_t(X, Y):
    K, N, d = X.shape
    A_list = np.zeros((N-1, d, d))
    b_list = np.zeros((N-1, d))
    Q_list = np.zeros((N-1, d, d))
    R2_list = np.zeros(N-1)
    reg_list = []

    for t in range(1, N):  
        S_t = Y[:, t-1, :]  
        Z_t = X[:, t,   :] 
        reg_t = LinearRegression(fit_intercept=True)
        reg_t.fit(S_t, Z_t)
        A_t = reg_t.coef_      # (d, d)
        b_t = reg_t.intercept_ # (d,)
        Z_hat_t = reg_t.predict(S_t)
        R_t = Z_t - Z_hat_t
        Q_t = np.cov(R_t, rowvar=False, bias=False)
        r2 = reg_t.score(S_t, Z_t)

        A_list[t-1, :, :] = A_t
        b_list[t-1, :] = b_t
        Q_list[t-1, :, :] = Q_t
        R2_list[t-1] = r2
        reg_list.append(reg_t)
    params = {"A": A_list,"b": b_list,"Q": Q_list,"regs": reg_list,"R2": R2_list}
    return params

In [7]:
def fit_conditional(items, items_z, population, n_selected, n_obj, n_con, rng, if_inital):
    objectives, objectives_z, cumu_objectives, cumu_objectives_z = get_norm_cumu_objectives(items, items_z, population, 
                                                                                                n_selected, n_obj, n_con, 
                                                                                                rng, if_inital)
    dist_params = fit_markov_in_y_by_t(objectives_z, cumu_objectives_z)
    return objectives_z, dist_params

In [23]:
def conditional_density_given_Y_and_t(X_candidates, y_normal, params_time, t):
    A_all = params_time["A"]  
    b_all = params_time["b"]  
    Q_all = params_time["Q"]  

    A_t = A_all[t-1]
    b_t = b_all[t-1]
    Q_t = Q_all[t-1]
    X_candidates = np.asarray(X_candidates)
    y_normal = np.asarray(y_normal).reshape(-1)
    mean_t = A_t @ y_normal + b_t
    Q_t = Q_t + 1e-1 * np.eye(Q_t.shape[0])

    densities = mvn.pdf(X_candidates, mean=mean_t, cov=Q_t)
    return densities

In [20]:
def base_rate_model(items_z, XX_0):
    mean0 = np.mean(XX_0, axis = 0)
    Sigma0 = np.cov(XX_0.T) 
    # add regularization to diagonal for singularity
    Sigma0 = Sigma0 + 1e-1 * np.eye(Sigma0.shape[0])
    mvn0 = mvn(mean=mean0, cov=Sigma0)
    x_candidates = items_z
    probabilities = mvn0.pdf(x_candidates)
    probabilities = (probabilities+1e-12)/sum(probabilities+1e-12)
    return probabilities

In [18]:
def sample_population_conditional(
    samples, samples_z, objectives_z, dist_params,
    pop_size, n_selected, capacity, rng): # no use of rng

    pop_count = 0
    population = np.zeros((pop_size, n_selected), dtype=np.int32)
    n_items = samples.shape[0]

    while pop_count < pop_size:
        # select sequentially
        knapsack = np.zeros(n_selected, dtype=int) # here knapsack is knapsack indices
        for n in range(n_selected):
            if n == 0: # select first item
                probabilities = base_rate_model(samples_z, objectives_z[:, 0, :])
                first_choice = rng.choice(n_items, p=probabilities)
                first_item = samples_z[first_choice,:]
                x_indices = np.setdiff1d(np.arange(n_items), knapsack)
                y_prev = first_item 
                knapsack[0] = first_choice
            else:
                x_indices = np.setdiff1d(np.arange(n_items), knapsack[:n])
                x_candidates = samples_z[x_indices, :]
                densities = conditional_density_given_Y_and_t(
                    x_candidates, y_prev, dist_params, n
                )
                probabilities = densities/sum(densities)
                next_choice = rng.choice(len(probabilities), p=probabilities)
                next_index = x_indices[next_choice]
                next_item = samples_z[next_index,:]
                knapsack[n] = next_index
                y_prev = y_prev + next_item
        
        constraint = np.sum(samples[knapsack, -1])
        if constraint <= capacity:
            population[pop_count, :] = knapsack
            pop_count += 1
    
    return population

In [11]:
# # normalize y during sampling
# def sample_population_conditional(
#     samples, samples_z, objectives_z, dist_params, ecdf_table, 
#     pop_size, n_selected, capacity, rng):

#     pop_count = 0
#     population = np.zeros((pop_size, n_selected), dtype=np.int32)
#     n_items = samples.shape[0]

#     while pop_count < pop_size:
#         knapsack = np.zeros(n_selected, dtype=int)   
#         y_prev_orig = None 
#         y_prev_z = None  
#         for n in range(n_selected):
#             if n == 0: 
#                 probabilities = base_rate_model(samples_z, objectives_z[:, 0, :])
#                 first_choice = rng.choice(n_items, p=probabilities)
#                 knapsack[0] = first_choice
#                 y_prev_orig = samples[first_choice, :] 
#                 y_prev_z = samples_z[first_choice, :]
#             else:
#                 x_indices = np.setdiff1d(np.arange(n_items), knapsack[:n])
#                 x_candidates = samples_z[x_indices, :]
#                 if n == 1:
#                     current_predictor_z = y_prev_z
#                 else:
#                     t = n - 1
#                     current_predictor_z = np.empty_like(y_prev_orig)
#                     for i in range(len(y_prev_orig)):
#                         ranks = np.searchsorted(ecdf_table[t, i, :], y_prev_orig[i], side='right')
#                         u = ranks / len(ecdf_table[t, i, :])
#                         u = np.clip(u, 1e-12, 1 - 1e-12)
#                         current_predictor_z[i] = norm.ppf(u)

#                 densities = conditional_density_given_Y_and_t(
#                     x_candidates, current_predictor_z, dist_params, n
#                 )
                
#                 probabilities = densities/sum(densities)
#                 next_choice = rng.choice(len(probabilities), p=probabilities)
#                 next_index = x_indices[next_choice]
#                 knapsack[n] = next_index
#                 y_prev_orig = y_prev_orig + samples[next_index, :]
        
#         constraint = np.sum(samples[knapsack, -1])
#         if constraint <= capacity:
#             population[pop_count, :] = knapsack
#             pop_count += 1
    
#     return population


In [12]:
class KnapsackEDACond:
    def __init__(self, items, capacity, n_selected, n_obj, n_con, pop_size=1000, 
                 generations=5, max_no_improve_gen=20, seed=1123):
        self.items = items
        self.items_z = None
        self.capacity = capacity
        self.n_selected = n_selected
        self.n_obj = n_obj
        self.n_con = n_con
        self.pop_size = pop_size
        self.generations = generations
        self.max_no_improve_gen = max_no_improve_gen
        self.rng = random.default_rng(seed=seed)
        self.if_inital = True
        self.ecdf_table = None
        
        # self.distribution = None
        self.first_item_dist = None
        self.distribution_params = None
        self.selected_population = None  # (pop_size, n_selected)
        self.selected_objectives = None  # (pop_size, n_obj) objective values are summed over solutions
        self.objectives_z = None  # (pop_size, n_selected, n_obj)

        self.distribution_params_table = []
        self.pareto_indices_table = []
        self.pareto_front_table = []
        self.js_div_list = []
        
    def _generate_initial_population(self):
        n_items = self.items.shape[0]
        distribution = np.ones(n_items) / n_items
        population = sample_population(
            self.items, distribution, self.pop_size, self.n_selected, 
            self.capacity, self.rng
        )
        objectives = get_objectives(self.items, population, self.n_obj)
        
        ranks, fronts = non_dominated_sort(objectives)
        distances_all_solutions = np.zeros(population.shape[0], dtype=float)
        for f in fronts:
            distances = assign_crowding_distance(objectives[f, :])
            distances_all_solutions[f] = distances
        
        select_indices = np.array([], dtype=int)
        while len(select_indices) < self.pop_size:
            indice = binary_tournament_selection(
                population, ranks, distances_all_solutions, self.rng
            )
            select_indices = np.concatenate([select_indices, np.array([indice])])
        
        selected_population = population[select_indices]
        selected_objectives = objectives[select_indices]

        _, _, self.items_z = transform_items_to_z(self.items)
        self.objectives_z, self.distribution_params = fit_conditional(self.items, self.items_z, selected_population, 
                                                                        self.n_selected, self.n_obj, self.n_con,
                                                                        self.rng, self.if_inital) # may need a different rng
        self.first_item_dist = base_rate_model(self.items_z, self.objectives_z[:, 0, :])
        self.selected_population = selected_population
        self.selected_objectives = selected_objectives
        
        return #selected_population, selected_objectives, items_z, objectives_z, distribution_params
    
    def _update_distribution(self):
        population = sample_population_conditional(
            self.items, self.items_z, self.objectives_z, self.distribution_params, 
            self.pop_size, self.n_selected, self.capacity, self.rng
        )
        objectives = get_objectives(self.items, population, self.n_obj)
        
        # Find current pareto front
        # _, _, _, fronts_current = non_dominated_sort(objectives)
        _, fronts_current = non_dominated_sort(objectives)
        pareto_indices = population[fronts_current[0]]
        
        objectives = np.vstack((self.selected_objectives, objectives))
        population = np.vstack((self.selected_population, population))
        
        # _, _, ranks, fronts = non_dominated_sort(objectives)
        ranks, fronts = non_dominated_sort(objectives)
        select_indices = np.array([], dtype=np.int32)
        for f in fronts:
            if len(select_indices) + len(f) <= self.pop_size:
                select_indices = np.concatenate([select_indices, f])
            else:
                remaining_size = self.pop_size - len(select_indices)
                f_distance = assign_crowding_distance(objectives[f, :])
                sort_indices = np.argsort(f_distance)[::-1]
                remaining = f[sort_indices[:remaining_size]]
                select_indices = np.concatenate([select_indices, remaining])
                break
        
        selected_population = population[select_indices]
        selected_objectives = objectives[select_indices]
        n_training = int(self.pop_size*0.15)
        training_population = selected_population[:n_training]
        
        # update distribution
        self.objectives_z, self.distribution_params = fit_conditional(self.items, self.items_z, training_population, 
                                                                        self.n_selected, self.n_obj, self.n_con,
                                                                        self.rng, self.if_inital)
        
        self.selected_population = selected_population
        self.selected_objectives = selected_objectives

        # check distribution convergence
        updated_first_item_dist = base_rate_model(self.items_z, self.objectives_z[:, 0, :])
        self.first_item_dist[self.first_item_dist < 1E-08] = 1E-08
        updated_first_item_dist[updated_first_item_dist < 1E-08] = 1E-08
        js_div = jensenshannon(self.first_item_dist, updated_first_item_dist)**2
        self.first_item_dist = updated_first_item_dist
                                                                        
        return pareto_indices, js_div

    def run(self):
        self._generate_initial_population()
        self.if_inital = False
        
        # Run generations (fixed number of generations)
        for g in range(self.generations):
            print(f"Generation {g+1}/{self.generations}")
            pareto_indices, js_div = self._update_distribution()
            print(f"number of front 0: {pareto_indices.shape[0]}")
            
            pareto_front = np.zeros((pareto_indices.shape[0], self.items.shape[1]))
            for k in range(pareto_indices.shape[0]):
                pareto_front[k, :] = np.sum(self.items[pareto_indices[k, :], :], axis=0)
                
            self.distribution_params_table.append(self.distribution_params.copy())
            self.pareto_indices_table.append(pareto_indices.copy())
            self.pareto_front_table.append(pareto_front.copy())
            self.js_div_list.append(js_div)

        return {
            'distribution_params_table': self.distribution_params_table,
            'pareto_indices_table': self.pareto_indices_table,
            'pareto_front_table': self.pareto_front_table,
            'js_div_list': self.js_div_list,
            'objectives_z': self.objectives_z,
            'items_z': self.items_z
        }
        
        # # Run generations (until convergence)
        # no_improve_gen = 0
        # prev_js_div = None
        # generation = 0
        # while no_improve_gen < self.max_no_improve_gen:
        #     generation += 1
        #     print(f"Generation {generation} (no improve count: {no_improve_gen})")
        #     pareto_indices, js_div = self._update_distribution()
        #     print(f"number of front 0: {pareto_indices.shape[0]}")

        #     pareto_front = np.zeros((pareto_indices.shape[0], self.items.shape[1]))
        #     for k in range(pareto_indices.shape[0]):
        #         pareto_front[k, :] = np.sum(self.items[pareto_indices[k, :], :], axis=0)
                
        #     self.distribution_params_table.append(self.distribution_params.copy())
        #     self.pareto_indices_table.append(pareto_indices.copy())
        #     self.pareto_front_table.append(pareto_front.copy())
        #     self.js_div_list.append(js_div)
                
        #     if prev_js_div is not None:
        #         diff = prev_js_div - js_div
        #         if np.abs(diff) > 0.0001: # lowered criteria
        #             no_improve_gen = 0
        #         else:
        #             no_improve_gen += 1
        #     else:
        #         no_improve_gen = 0
        #     prev_js_div = js_div

        # return {
        #     'distribution_params_table': self.distribution_params_table,
        #     'pareto_indices_table': self.pareto_indices_table,
        #     'pareto_front_table': self.pareto_front_table,
        #     'js_div_list': self.js_div_list
        # }

In [13]:
# items_seed = 1211
# # Generate data
# items, rpos = generate_example_data(r, shape, scale, n_items=n_items, seed=items_seed)

In [12]:
# a test case
kn = loadmat('/home/tailai/data/knapsack/runB/kn_1_1_allneg_60_6_3.mat')
items = kn['items'][1]
shape = kn['shape']
scale = kn['scale']

In [13]:
#parameters
eda_seed = 1223
n_items = 60
n_selected = 6
n_obj = 3
n_con = 1
capacity = int(shape[-1]*scale[-1]*n_selected)
pop_size = 1500
generations = 50 # do not matter if check convergence
max_no_improve_gen = 10

In [16]:
# Run EDA
eda = KnapsackEDACond(
    items=items,
    capacity=capacity,
    n_selected=n_selected,
    n_obj=n_obj,
    n_con=n_con,
    pop_size=pop_size,
    generations=generations,
    max_no_improve_gen=max_no_improve_gen,
    seed=eda_seed
)

#organize results    
results = eda.run()
with open('results_cond.pkl', 'wb') as f:
    pickle.dump(results, f)

Generation 1/50
number of front 0: 32
Generation 2/50
number of front 0: 42
Generation 3/50
number of front 0: 61
Generation 4/50
number of front 0: 49
Generation 5/50
number of front 0: 63
Generation 6/50
number of front 0: 67
Generation 7/50
number of front 0: 74
Generation 8/50
number of front 0: 84
Generation 9/50
number of front 0: 69
Generation 10/50
number of front 0: 88
Generation 11/50
number of front 0: 86
Generation 12/50
number of front 0: 95
Generation 13/50
number of front 0: 87
Generation 14/50
number of front 0: 80
Generation 15/50
number of front 0: 80
Generation 16/50
number of front 0: 70
Generation 17/50
number of front 0: 81
Generation 18/50
number of front 0: 84
Generation 19/50
number of front 0: 74
Generation 20/50
number of front 0: 122
Generation 21/50
number of front 0: 144
Generation 22/50
number of front 0: 238
Generation 23/50
number of front 0: 270
Generation 24/50
number of front 0: 416
Generation 25/50
number of front 0: 519
Generation 26/50
number of f

In [14]:
import pickle
with open('results_cond.pkl', 'rb') as f:
    results = pickle.load(f)

In [15]:
dist_params = results['distribution_params_table'][-1]
pareto_solutions = results['pareto_indices_table'][-1]
objectives_z = results['objectives_z']
items_z = results['items_z']

In [16]:
def converged_pf_from_dist(
    dist_params, items, items_z, objectives_z, pareto_solutions, 
    capacity, n_selected, n_obj, f_seed=1234, 
    sample_size=1000, max_iters=100, max_no_change=2):

    rng = np.random.default_rng(f_seed)       
 
    # if len(pareto_solutions) > 0:
    pareto_solutions = np.unique(np.sort(pareto_solutions, axis=1), axis=0)
    pareto_objectives = get_objectives(items, pareto_solutions, n_obj)
    # else:
        # pareto_solutions = np.empty((0, n_selected), dtype=int)
        # pareto_objectives = np.empty((0, n_obj))

    no_change = 0
    counter = 0
    while no_change < max_no_change and counter < max_iters:
        new_sample = sample_population_conditional(
            items, items_z, objectives_z, dist_params,
            sample_size, n_selected, capacity, rng
        )
        
        # if len(pareto_solutions) == 0:
        #      all_solutions = np.unique(np.sort(new_sample, axis=1), axis=0)
        # else:
        all_solutions = np.unique(np.sort(np.vstack((pareto_solutions, new_sample)), axis=1), axis=0)
        all_objectives = get_objectives(items, all_solutions, n_obj)
        nd_idx = non_dominated(all_objectives)
        nd_idx = nd_idx.astype(bool)
        new_pareto_solutions = all_solutions[nd_idx]
        new_pareto_objectives = all_objectives[nd_idx]
        
        if np.array_equal(np.unique(new_pareto_objectives, axis=0), np.unique(pareto_objectives, axis=0)):
            no_change += 1
        else:
            no_change = 0

        pareto_solutions, pareto_objectives = new_pareto_solutions, new_pareto_objectives
        counter += 1
        print(f"iter {counter}: {len(pareto_solutions)}")
    
    return pareto_solutions, pareto_objectives, counter

In [24]:
pareto_solutions, pareto_objectives, counter = converged_pf_from_dist(
    dist_params, items, items_z, objectives_z, pareto_solutions, 
    capacity, n_selected, n_obj, 
    max_no_change=5)

iter 1: 112
iter 2: 111
iter 3: 113
iter 4: 112
iter 5: 119
iter 6: 115
iter 7: 121
iter 8: 123
iter 9: 124
iter 10: 125
iter 11: 125
iter 12: 126
iter 13: 127
iter 14: 128
iter 15: 129
iter 16: 129
iter 17: 131
iter 18: 131
iter 19: 132
iter 20: 132
iter 21: 132
iter 22: 131
iter 23: 131
iter 24: 133
iter 25: 134
iter 26: 134
iter 27: 134
iter 28: 134
iter 29: 134
iter 30: 132
iter 31: 132
iter 32: 132
iter 33: 132
iter 34: 132
iter 35: 132


In [25]:
from hdf5storage import loadmat, savemat
kn= loadmat('/home/tailai/data/knapsack/runB/kn_1_1_allneg_60_6_3.mat')
pareto_front_final = kn['pareto_front_final'][1]

In [26]:
obj1 = pareto_objectives
obj2 = pareto_front_final[:, :3]

def view1D(a, b):
    a = np.ascontiguousarray(a)
    b = np.ascontiguousarray(b)
    void_dt = np.dtype((np.void, a.dtype.itemsize * a.shape[1]))
    return a.view(void_dt).ravel(),  b.view(void_dt).ravel()

A, B = view1D(obj1, obj2)
common_rows = np.intersect1d(A, B)
count = len(common_rows)
print(f"Number of shared identical rows: {count}")

Number of shared identical rows: 37


In [27]:
from pymoo.indicators.hv import HV
A = obj1.astype(np.float64)
B = obj2.astype(np.float64)
A_min = -A
B_min = -B

worst_min = np.max(np.vstack([A_min, B_min]), axis=0)
ref = worst_min * 1.05

hv = HV(ref_point=ref)

A_hv = hv(A_min)
B_hv = hv(B_min)

print(A_hv)
print(B_hv)
print((A_hv-B_hv)/B_hv)

60267.64012499999
73943.21762499999
-0.1849470166331567
